# Importando las librerías requeridas

In [5]:
import os
import re
import numpy as np
from collections import Counter
from pathlib import Path
import pandas as pd

# Definiendo variables del entorno de trabajo

In [6]:
basepath = Path("/home/cmejia3")
datasetIn =basepath / "datasets/papers-txt/"
datasetOut =basepath / "datasets/salidas/"

# Funciones para preprocesar los archivos


In [7]:
def fileSize(fileIn):
    size=os.stat(fileIn).st_size
    return size

def wordsCount(contenido):
    totWwords=contenido.split()
    return len(totWwords)

def cleanWordCount(contenido,datasetOut):
    contenido =re.sub('(f|ht)tp(s?)://(.*)[.][a-z]+',' ',contenido) # Eliminar las URL
    #contenido =re.sub('REFERENCES (\S|\w)+',' ',contenido) # Eliminar la bibliografia
    contenido =re.sub('[a-zA-Z0-9.?{}]+@\w+\.\w+.\w*',' ',contenido) # Eliminar los correos
    contenido =re.sub('\[[a-zA-Z0-9\,\. ]+\]',' ',contenido) # Eliminar cualquier contenido entre corchetes
    contenido =re.sub('\([a-zA-Z0-9\,\.\- ]+\)',' ',contenido) # Eliminar cualquier contenido entre paréntesis
    contenido =re.sub('((et al\.)|(i\.i\.d\.)|(i\.e\.)|\-|\'|\’|\`)',' ',contenido) # Eliminar abreviaciones, apostrofes y guion
    #contenido =re.sub('(f|F)igure [0-9]+.[0-9]',' ',contenido) # Eliminar Figure
    contenido =re.sub('[^a-zA-Z_á\éíóúà\èìòùäëïöü\s]',' ',contenido) # Eliminar caracteres que no sean: letra, número o vocales acentuadas
    contenido =re.sub(' +',' ',contenido) # Eliminar espacios en blanco
    contenido =re.sub('(a-z|A-Z){1,1}',' ',contenido) # Eliminar palabras o números de un caracter de longitud   
    #contenido =re.sub('[^A-Za-z0-9.,_%+-\(\)\[\]\´\'\`]',' ',contenido)
    #contenido =re.sub('\[(0-9)+\]',' ',contenido)    
    #totWordDepurado = Counter(map(str, contenido.split()))
    outputFile= open(datasetOut, 'w', encoding='UTF-8')
    outputFile.write(contenido)
    outputFile.close()
    totWwords=contenido.split()
    #print("Total de palabras {}".format(len(totWwords)))
    #print("Total de palabras después del pre-procesamiento: {}".format(totWordDepurado))
    return len(totWwords)

# Pre-procesamiento de archivos

In [8]:
fileSummary = datasetOut / "CleanSummary.csv"

contenido= "Archivo" + ";" + "Tamaño(K)" + ";" + "Cant Palabras Inicial" + ";" + "Cant Palabras depuradas"+ ";" +"Porc Limpieza"+ "\n"
resultado = pd.DataFrame()
for f in datasetIn.glob('*.txt'):
    tmpSize=round(fileSize(f)/1014)
    input_file = open(f, "r", encoding = 'utf-8')
    input_aux = input_file.read()
    tmpWordsOri=wordsCount(input_aux)
    out = datasetOut / str(f).split('/')[-1]
    tmpWordsEnd=cleanWordCount(input_aux,out)
    tmpPerClean=round((tmpWordsEnd/tmpWordsOri)*100)
    aux= pd.DataFrame({'Archivo': str(f).split('/')[-1], 'Tamaño(K)': [tmpSize], 'Cant Palabras Inicial': [tmpWordsOri],'Cant Palabras depuradas': [tmpWordsEnd],"Porc Limpieza":[tmpPerClean]})
    resultado = pd.concat([resultado, aux])

resultado.to_csv(fileSummary, sep = ';', index = False)
